In [1]:
import pandas as pd
import numpy as np

In [2]:
import torch
torch.cuda.is_available()

False

In [3]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import FARMReader, TransformersReader
from haystack.pipelines import ExtractiveQAPipeline

#haystack contains a search system for retrieval and QA across documents.
#designed for large documents, but pipeline also works for single document QA 

INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


In [4]:
# In-Memory Document Store
from haystack.document_stores import InMemoryDocumentStore
document_store = InMemoryDocumentStore() #to enable documents stored in local memory

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


In [5]:
doc_dir = "testdata"

dicts = convert_files_to_dicts(dir_path=doc_dir, split_paragraphs=True)

## if there are multiple documents for database, this puts all relevant docs into dictionary

INFO - haystack.utils.preprocessing -  Converting testdata\s12967-020-02620-5 (1).txt


In [6]:
dicts

[{'content': 'Hou et al. J Transl Med\t(2020) 18:462\nhttps://doi.org/10.1186/s12967-020-02620-5\n               \nJournal of Translational Medicine\n            ',
  'meta': {'name': 's12967-020-02620-5 (1).txt'}},
 {'content': 'Predicting 30-days mortality for MIMIC-III patients with sepsis-3: a machine learning approach using XGboost\nNianzong Hou1†, Mingzhe Li2, Lu He3, Bing Xie1, Lin Wang4, Rumin Zhang4, Yong Yu4, Xiaodong Sun5, Zhengsheng Pan6 and Kai Wang4*†',
  'meta': {'name': 's12967-020-02620-5 (1).txt'}},
 {'content': '\n*Correspondence: wangkaiicu@163.com\n†Nianzong Hou and Kai Wang contributed equally to this work\n4 Department of Critical Care Medicine, Zibo Central Hospital, Shandong First Medical University , Zibo 255036, Shandong, China\nFull list of author information is available at the end of the article',
  'meta': {'name': 's12967-020-02620-5 (1).txt'}},
 {'content': 'Background\nSepsis is a common and economically significant disease which has become an importan

In [7]:
##now writes the dict to haystack document store

document_store.write_documents(dicts)

INFO - haystack.document_stores.base -  Duplicate Documents: Document with id '911f0a35aac5f8a66a32093438b38afa' already exists in index 'document'


In [8]:
# An in-memory TfidfRetriever based on Pandas dataframes
# retrievers narrow down Reader scope to smaller text units
# see haystack documentation -> other retrievers

from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

INFO - haystack.nodes.retriever.sparse -  Found 51 candidate paragraphs from 51 docs in DB


In [9]:
# Reader scans text returned by retriever and extracts k-best answers
# Load a fine-tuned  model (e.g. RoBERTa QA = "deepset/roberta-base-squad2")
# alternatives (Reader): TransformersReader (leveraging the pipeline of the Transformers package)
# alternatives (Models): e.g. "distilbert-base-uncased-distilled-squad" (fast) or "deepset/bert-large-uncased-whole-word-masking-squad2" (good accuracy)
# can adjust the model to return "no answer possible" with the no_ans_boost. Higher values mean the model prefers "no answer possible"
# alternatively, QA models on model hub (https://huggingface.co/models)
#sota: ahotrod/albert_xxlargev1_squad2_512
#dmis-lab/biobert-large-cased-v1.1-squad
#

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 15 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
INFO - haystack.modeling.infer -  /w\  /w\  /w\  /w\ 

In [10]:
# Alternative example:
# reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

In [11]:
#sets pipeline to contain retriever and reader

pipe = ExtractiveQAPipeline(reader, retriever)

In [12]:
q1="what was the study objective?"
q2="what disease is being studied?"
q3="how many patient data samples were included in this study?"
#q4="how many data samples were used to train the model?"
#q5="what modalities of data is used in this study?"
q6="what are the variable types used in this study?"
q7="what country was the study conducted in?"
q8="what city did the data come from?"
q9="did data come from an existing database?"
q10="did data come from an organisation?"
q11="was the model tested on a independent dataset?"
q12="what was the outcome measured by the area under the curve (AUC)?"

In [ ]:
# Number of candidates the reader and retriever return
# Higher top_k for retriever = better accuracy (but slower)
qlist = [q1, q2, q3, q6, q7, q8, q9, q10, q11, q12]
plist = qlist.copy() #keep same length
l = len(qlist)

for i in range(0,l):
    plist[i] = pipe.run(
            query=qlist[i], params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
        )

#p1 = pipe.run(
#    query=q1, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
#)

Inferencing Samples:   0%|                                                                 | 0/1 [00:00<?, ? Batches/s]C:\Users\Joe Z\anaconda3\envs\pytorch\lib\site-packages\haystack\modeling\model\prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples:   0%|                                                                 | 0/1 [00:00<?, ? Batches/s]C:\Users\Joe Z\anaconda3\envs\pytorch\lib\site-packages\haystack\modeling\model\prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds tow

Inferencing Samples:   0%|                                                                 | 0/1 [00:00<?, ? Batches/s]C:\Users\Joe Z\anaconda3\envs\pytorch\lib\site-packages\haystack\modeling\model\prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples:   0%|                                                                 | 0/1 [00:00<?, ? Batches/s]C:\Users\Joe Z\anaconda3\envs\pytorch\lib\site-packages\haystack\modeling\model\prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds tow

Inferencing Samples:   0%|                                                                 | 0/1 [00:00<?, ? Batches/s]C:\Users\Joe Z\anaconda3\envs\pytorch\lib\site-packages\haystack\modeling\model\prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples:   0%|                                                                 | 0/1 [00:00<?, ? Batches/s]

In [ ]:
for i in range(0,l):
    print_answers(plist[i], details='minimum')